In [ ]:
import requests

import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
import torch
from transformers import (SamModel, SamProcessor)

from utils import *

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

GPU = -1

device = torch.device("cpu")
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device).eval()
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

In [ ]:
img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")

input_points = [[[500, 600]]]

inputs = processor(raw_image, input_points=input_points, return_tensors="pt").to(device)

In [ ]:
model_dynamic_quantized = torch.quantization.quantize_dynamic(
    model, qconfig_spec={torch.nn.Linear, torch.nn.Conv2d},
    dtype=torch.qint8)

In [ ]:
model_dynamic_quantized.eval()
with torch.no_grad():
    image_embeddings = model_dynamic_quantized.get_image_embeddings(inputs["pixel_values"])
    inputs.pop("pixel_values", None) # pixel_values are no more needed
    inputs.update({"image_embeddings": image_embeddings})
    outputs = model_dynamic_quantized(**inputs)

In [ ]:
masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())[0]
scores = outputs.iou_scores
show_masks_on_image(raw_image, masks, scores)

In [ ]:
model.eval()
with torch.no_grad():
    image_embeddings = model.get_image_embeddings(inputs["pixel_values"])
    inputs.pop("pixel_values", None) # pixel_values are no more needed
    inputs.update({"image_embeddings": image_embeddings})
    outputs = model(**inputs)

In [ ]:
masks = processor.image_processor.post_process_masks(outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(), inputs["reshaped_input_sizes"].cpu())[0]
scores = outputs.iou_scores
show_masks_on_image(raw_image, masks, scores)

In [ ]:
scripted_quantized_model = torch.jit.script(model_dynamic_quantized)
scripted_quantized_model.save("bin/sam_scripted_quantized.pt")